In [1]:
import scipy.sparse as spr
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
p = pv.BackgroundPlotter()

In [28]:
Th = fem.square( 2, 1, he = 1/100, element_order=2, structured = True, quads=False, save=False)

In [29]:
Th

< mesh object with 80601 nodes, 40604 elements of types ( point1, line3, tri6 ) >

In [3]:
Th.group_names.keys()

dict_keys(['topleft', 'bottomleft', 'topright', 'bottomright', 'left', 'bottom', 'right', 'top', 'domain'])

In [18]:
Th.group_names['topleft']['members']

[{'types': array([15], dtype=int32),
  'tags': [array([1], dtype=uint64)],
  'nodes': array([1], dtype=uint64),
  'indices': [array([[0]], dtype=uint64)]}]

In [12]:
max(Th.y.real)

array([0.5])

In [30]:
f = oti.sin(Th.x*Th.y)**2

In [35]:
def int2d(f, intorder = 1, ndim_an  = 2):
    
    els = Th.elements[2]

    integral = oti.zero()
    
    #integrate 2D a fem array.
    for j in range(els['types'].size):

        element = fem.element[ els['types'][j] ]

        if not element.is_allocated():
            element.end()
            element.allocate(intorder)
            element.allocate_spatial(ndim_an,compute_Jinv = True)

            # Here the temporals shhould be created.
            fh = oti.zeros( ( element.nbasis, 1 ) )
        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):

            #  print('Element indices:',type(els['nodes'][j][i][0]))
            element.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
            element.compute_jacobian()

            for ii in range(element.nbasis):
                fh[ii,0] = f[ int(elm_nodes[i,ii]),0]
            #end for 

            integral = integral + oti.gauss_integrate(oti.dot_product(fh,element.N),element.w_dJ)
#             integral = integral + oti.dot_product( fh, oti.gauss_integrate( element.N, element.w_dJ) )

        # end for 
    # end for.
    
    fem.end_elements()
    
    return integral


In [36]:
%timeit int2d(f)

1.13 s ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
els = Th.elements[2]

intorder = 1
ndim_an  = 2

# element = fem.tri3
# element.end()
# element.allocate(intorder)
# element.allocate_spatial(ndim_an,compute_Jinv = True)

integral = 0


#integrate 2D a fem array.
for j in range(els['types'].size):

    element = fem.element[ els['types'][j] ]
    
    if not element.is_allocated():
        element.end()
        element.allocate(intorder)
        element.allocate_spatial(ndim_an,compute_Jinv = True)

        # Here the temporals shhould be created.
        fh = oti.zeros( ( element.nbasis,1) )
    # end if 
    
    elm_nodes = els['indices'][j]
    
    for i in range(elm_nodes.shape[0]):
        
        #  print('Element indices:',type(els['nodes'][j][i][0]))
        element.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
        element.compute_jacobian()
        
        for ii in range(element.nbasis):
            fh[ii,0] = f[ int(elm_nodes[i,ii]),0]
        #end for 
        
        # integral = integral + oti.gauss_integrate(oti.dot_product(fh,element.N),element.w_dJ)
        integral = integral + oti.dot_product( fh, oti.gauss_integrate( element.N, element.w_dJ) )
        
    # end for 
    
# end for.
fem.end_elements()
integral.real

0.05349579666750341

In [20]:
els['types']

array([3], dtype=int32)

In [16]:
not element.is_initialized()

False

In [60]:
max(f.real)

array([0.22984885])

In [61]:
grid = Th.to_pv(dims=2,pd=[f.real],pd_names=["function"])

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=16,cmap='jet')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7ff322834888

In [51]:
Th.elements[2]

{'types': array([3], dtype=int32),
 'tags': [array([ 305,  306,  307, ..., 5302, 5303, 5304], dtype=uint64)],
 'nodes': [array([[   0,    4,  300,  299],
         [ 299,  300,  301,  298],
         [ 298,  301,  302,  297],
         ...,
         [5148,  149,  150, 5149],
         [5149,  150,  151, 5150],
         [5150,  151,    3,  152]], dtype=uint64)]}